In [2]:
import requests
import pandas as pd

# Login

In [3]:
# roda acesso, api/
url = "http://127.0.0.1:9000/api/".replace("\\","/")

In [4]:
username="testeuser@zeladorx.com.br"
password="TESte2026&"

body = {
    "username":username,
    "password":password
}

In [5]:
response = requests.post("http://127.0.0.1:9000/api/login", body)

In [6]:
# dados do usuário logado
response.json()

# {
#     'token': '171db50f8cc06dae8ae7338bdc5048ddc97f6784', #token de acesso
#     'id_random': '6uW6BnjPFHyd', #id randomico unico
#     'user_id': 1, #id de chave primaria
#     'username': 'zeladorx', #username
#     'email': 'zeladorx@zeladorx.com.br', #email
#     'is_superuser': True, #é superusario do ecossitema contratado
# }

{'token': '92ca37bb8f53bb9a8fbaefad1155c6ab55677c11',
 'id_random': '6uW6BnjPFHyd',
 'user_id': 1,
 'username': 'zeladorx',
 'email': 'zeladorx@zeladorx.com.br',
 'is_superuser': True}

# Auth

In [7]:
# todas as rotas precisam do tokem para ser acessada
headers = {
    "Authorization": f"Token {response.json()['token']}"
}

# Fato servico

In [8]:
# retorna todas os acompanhamentos ja criados
# codigo de permissão necessário --> 390: Pode visualizar o detalhamento de serviços
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/ListFatoServicoLimpezaPredial/", headers=headers)
response.json()
df = pd.DataFrame(response.json())
display(df)

,id,id_random,Servico,data_hora_chegada_na_area,data_hora_retorno_area,Gerente,foto_entrega
0,1,B5dpHvdWTtdi,3,2025-08-18T16:59:00-03:00,2025-08-18T16:59:00-03:00,1,None
1,2,fl7yCqlYSkU0,4,2025-08-18T17:00:00-03:00,2025-08-18T17:00:00-03:00,1,None
2,4,xRYpzJQwKsPe,1,2025-08-18T17:02:00-03:00,2025-08-18T17:02:00-03:00,1,None


## Criar novo acompanhemto

In [9]:
## As colunas ServicoJardinagemAgendado são chaves estrangeiras e precisam estar devidamente configuradas
## sendo obriagatorios

In [10]:
# Lista de empresas que podem ser enviadas pelo usuario
# nao precisa de permissao
print("Serviços agendados")
response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/ListServicoLimpezaPredialAgendadoAnotados/", headers=headers)
response.json()
df = pd.DataFrame(response.json()['results'])
display(df)
print()

Serviços agendados


,id,id_random,DescricaoDoServico,DataDeInicio,DataDeConclusao,TipoServico,Areas,ServicosEscalados,dias_diferenca,novo_status
0,1,UWoNKAzEeZRs,steste,2025-07-10T18:04:00-03:00,2025-07-10T18:04:00-03:00,Regular,1,[1],-38,Em andamento
1,3,HYRe7BemFKQQ,steste,2025-08-18T16:58:00-03:00,2025-08-19T16:58:00-03:00,Regular,1,[1],0,Em andamento
2,4,PJ5FbfNKlytn,steste,2025-08-21T16:59:00-03:00,2025-08-22T16:59:00-03:00,Regular,1,[1],2,Em andamento


In [11]:
body = {
    'Servico': 1, #vem de api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
    'data_hora_chegada_na_area': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'data_hora_retorno_area': "2025-08-08T18:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'Gerente': 2, # vem de api/gerentes/ListGerenteJardinagemFromForms/
}


In [12]:
# codigo de permissão necessário --> 324: Pode acompanhar serviços agendados
# essa rota rece o id random do servico que sera crado o acompanhamento, deve ser validado jnto com o id
response = requests.post("http://127.0.0.1:9000/api/servicos/fato/CreateFatoServicoLimpezaPredial/UWoNKAzEeZRs/", body, headers=headers)
response.json()

{'success': True,
 'message': 'FatoServicoLimpezaPredial criado com sucesso.',
 'data': {'id': 5,
  'id_random': 'PAW0EXCXXDPh',
  'Servico': 1,
  'data_hora_chegada_na_area': '2025-08-08T13:57:00-03:00',
  'data_hora_retorno_area': '2025-08-08T18:57:00-03:00',
  'Gerente': 2,
  'foto_entrega': None}}

# Detalhes acompanhamento

In [13]:
# essa rota recebe o id random da acompanahemnto como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# detalhes do acompanhemtno criado anteriormente, obeserve o id_random
# http://127.0.0.1:9000/api/servicos/fato/FatoServicoJardinagemDetail/id_random/
# codigo de permissão necessário --> 402: Pode visualizar checklists
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/FatoServicoLimpezaPredialDetail/AVZ3rObLKuH7/", headers=headers)
response.json()

{'detail': 'No FatoServicoLimpezaPredial matches the given query.'}

# Atualizar acompanhamento

In [14]:
# as chaves body da atualização é exatamente igual o body da criacao
# atualiando a descrição para acompanhamento teste
body = {
    'Servico': 1, #vem de api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
    'data_hora_chegada_na_area': "2025-08-08T13:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'data_hora_retorno_area': "2025-08-08T18:57:00-03:00", #agendamento para 11/08/2025 13:57 -- > ajusta o fuso horario :00-03:00
    'Gerente': 2, # vem de api/gerentes/ListGerenteJardinagemFromForms/
}

In [15]:
# essa rota recebe o id random do checklist como parametro de url, consultar em /api/checklist/ListCheckListJardinagem
# atualizando o checklist criada anteriormente, obeserve o id_random
# codigo de permissão necessário --> 401: Pode editar checklists
response = requests.put("http://127.0.0.1:9000/api/servicos/fato/FatoServicoLimpezaPredialUpdate/AVZ3rObLKuH7/", body, headers=headers)
response.json()

{'detail': 'acompanhamento de limpeza predial não encontrada.'}

# Se deletar o acompanhamento

In [16]:
# essa rota recebe o id random do acompanhemtno como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# codigo de permissão necessário --> 327 Pode excluir serviços concluidos
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/IfDeleteFatoServicoLimpezaPredial/AVZ3rObLKuH7/", headers=headers)
response.json()

{'detail': 'Objeto não encontrado.'}

# Deleção do acompanhamento

In [17]:
# essa rota recebe o id random do acompanhemtno como parametro de url, consultar em api/servicos/fato/ListFatoServicoJardinagem/
# codigo de permissão necessário --> 327 Pode excluir serviços concluidos
response = requests.delete("http://127.0.0.1:9000/api/servicos/fato/DeleteFatoServicoLimpezaPredial/AVZ3rObLKuH7/", headers=headers)
response.json()

{'detail': 'Objeto não encontrado.'}

# Endpoints Auxiliares

# acompanhamentos associado a um servico especifico

In [18]:
# essa rota recebe  id random de um servico como parametro, obtido em api/servicos/agendados/ListServicoJardinagemAgendadoAnotados/
# codigo de permissão 390: Pode visualizar o detalhamento de serviços
response = requests.get("http://127.0.0.1:9000/api/servicos/fato/ListFatoServicoLimpezaPredialByservico/UWoNKAzEeZRs/", headers=headers)
response.json()

[{'id': 4,
  'id_random': 'xRYpzJQwKsPe',
  'Servico': 1,
  'data_hora_chegada_na_area': '2025-08-18T17:02:00-03:00',
  'data_hora_retorno_area': '2025-08-18T17:02:00-03:00',
  'Gerente': 1,
  'foto_entrega': None},
 {'id': 5,
  'id_random': 'PAW0EXCXXDPh',
  'Servico': 1,
  'data_hora_chegada_na_area': '2025-08-08T13:57:00-03:00',
  'data_hora_retorno_area': '2025-08-08T18:57:00-03:00',
  'Gerente': 2,
  'foto_entrega': None}]